In [0]:
import zipfile
import re
import urllib
import os
import numpy as np
import pandas as pd
import argparse
import torch
import torchvision.models
import torchvision.transforms as transforms
from PIL import Image
import keras
import copy
import pickle
import random
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import os
import time
import shutil

In [0]:
len(os.listdir("drive/My Drive/Data/Images/"))

32129

In [0]:
np.random.seed(0)

In [0]:
# os.chdir('drive')
os.chdir('..')
os.chdir('content')

In [0]:
#RUN1

def prepare_image(image):
    """
    prepare_image accepts a PIL image as input and transforms it so as to fit into the input format of resnet-50.
    The dimensions of the output remain constant (224 x 224) so that a uniformity is maintained in the code
    """
    if image.mode != 'RGB':
        image = image.convert("RGB")
    Transform = transforms.Compose([
            transforms.Resize([224,224]),      
            transforms.ToTensor(),
            ])
    image = Transform(image)   
    image = image.unsqueeze(0)
    return image.to(device)

In [0]:
#RUN2

#making pdip csv with left more popular
data_files_path = "drive/My Drive/Data/Tweets/"
data_folder = [data_files_path + f for f in os.listdir(data_files_path) if "data_#" in f]

# Retweets and Likes

In [0]:
def totcount(arr1,arr2):

     """
    totcount(arr1,arr2) takes in two tweets arr1, and arr2 in json format, and returns a value according to the equation: 0.7*retweet_count + 0.3*favorite_count
    The two scalar values 0.7 and 0.3 were obtained after training the model with 3 different combinations of iterations: [(0.6, 0.4),(0.7,0.3), and (0.8, 0.2)]
    """
    
    if arr1=='tweet_1' or arr2=='tweet_1':
        return None
    else:

        rcfc1=arr1[arr1.find("retweet_count"):arr1.find("favorited")-3].split(" ")
        # print(rcfc1)
        rc1=rcfc1[1][:-1]
        # print(rc1)
        fc1=rcfc1[3]
        # print(fc1)
        tot1=(0.7)*int(rc1)+int(fc1)*(0.3)
        ind1 = arr1.find("id_str")
        img_name_1 = arr1[ind1+10:ind1+29]
        ind1 = arr1.find("media_url")+13
        ind2 = arr1.find("media_url_http")
        media_url1 = arr1[ind1:ind2-4]

        rcfc2=arr2[arr2.find("retweet_count"):arr2.find("favorited")-3].split(" ")
        # print(rcfc2)
        rc2=rcfc2[1][:-1]
        # print(rc2)
        fc2=rcfc2[3]
        # print(fc2)
        tot2=(0.7)*int(rc2)+int(fc2)*(0.3)
        ind2 = arr2.find("id_str")
        img_name_2 = arr2[ind2+10:ind2+29]
        ind1 = arr2.find("media_url")+13
        ind2 = arr2.find("media_url_http")
        media_url2 = arr2[ind1:ind2-4]

        return( (tot1, img_name_1, media_url1, tot2, img_name_2, media_url2))
        # return( ceil(tot1), ceil(tot2) )

In [0]:
os.chdir("drive/My Drive/Data/Tweets")

In [0]:
#Get a list of all tweet files: alltweets


alltweets=os.listdir()
datafile=[]
tweetfile=[]

for i in alltweets:
    if i[0:1]=='d':
         datafile.append(i) 
    elif i[0:1]=='t':
        tweetfile.append(i)
    else:
        print(i)

.DS_Store


In [0]:
os.chdir("..")
os.chdir("Tweets")

In [0]:
os.chdir("..")
os.chdir("Images")
img_list = os.listdir()
len(img_list)

32129

In [0]:
os.chdir("..")
os.chdir("Tweets")
os.getcwd()

'/content/drive/My Drive/Data/Tweets'

In [0]:
# Get the pdip_dataset with pairs of images in pdip_strict

k=0
count_lives = 0
pdip_strict = []
not_present = []
for i in tweetfile:
    arr1=[]
    arr2=[]
    # print(i)
    twt=pd.read_csv(i)
    for j in range(0,twt.shape[0]):
        x1temp = twt.iloc[j][0]
        x2temp = twt.iloc[j][1]
        tup = totcount(twt.iloc[j][0],twt.iloc[j][1])
        if tup==None: continue
        t1,img1_name,media_url1,t2,img2_name,media_url2 = tup[0],tup[1],tup[2],tup[3],tup[4],tup[5]
        
        # t1,img1_name,t2,img2_name=totcount(twt.iloc[j][0],twt.iloc[j][1])
        # arr1.append(t1)
        # arr2.append(t2)
        if img1_name == img2_name:
            count_lives += 1
        if t1==t2:
            k+=1
        flag = 0
        if (img1_name+".jpg") not in img_list:
            not_present.append([img1_name,media_url1])
            flag = 1
        if (img2_name+".jpg") not in img_list:
            not_present.append([img2_name,media_url2])
            flag = 1
        # print(flag)
        if flag == 1:
            continue
        # print(t1,t2)
        if t1 < t2:
            pdip_strict.append([img1_name,img2_name])
        elif t2 > t1:
            pdip_strict.append([img2_name,img1_name])
        elif t1 != 0:
            pdip_strict.append([img2_name,img1_name])

        
    # print(arr1)
    # print(arr2)


# print(k)

In [0]:

pdip_strict = np.asarray(pdip_strict)

np.random.shuffle(pdip_strict)

#change this according to your val

pdip_val = pdip_strict[10000:14000]

14596

In [0]:
# os.chdir("..")
# os.chdir("Images")
os.getcwd()

'/content/drive/My Drive/Data/Tweets'

Train Code

In [0]:
import os
import tweepy
import pandas as pd
from json import dump
from json import *
# !pip install wget
import wget
from datetime import date
import numpy as np
import csv

import re
import urllib
import zipfile

In [0]:
#Calculate the loss in every iteration to backpropagate

def loss_helper(P_ab, Phat_ab):
    l = -(Phat_ab*torch.log(P_ab)) - ((1-Phat_ab)*torch.log(1-P_ab))
    return torch.mean(l)

def my_loss(f_a,f_b,target):
    O_ab = f_a - f_b
    P_ab = torch.exp(O_ab)/(1+torch.exp(O_ab))
    return loss_helper(P_ab,target)

In [0]:
def train_model(model, optimizer, scheduler, num_epochs=25):

    """
    take the pretrained model, scheduler, number of epochs, and optimizer and input and perform forward propagation on the train and test set. after that calculate loss and accuracy for each and using the loss from train set, perform back propagation
    """

    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        model.train()

        running_loss = 0.0
        running_corrects = 0.0

        for i in range(len(train_data)):
            # im1 = prepare_image(train_data[i][0])
            # im2 = prepare_image(train_data[i][1])
            im1 = train_data[i][0]
            im2 = train_data[i][1]
            
            label = (float)(train_label[i])
            # print(label)
            optimizer.zero_grad()

            with torch.set_grad_enabled(True):
                op1 = (model(im1))
                op2 = (model(im2))
                if op1>op2:
                    output_label = 1.0
                else:
                    output_label = 0.0
                # loss = my_loss(op1,op2,output_label)
                loss = my_loss(op1,op2,label)

                loss.backward()
                optimizer.step()

                running_loss += loss.item()
                if output_label == label:
                    running_corrects += 1
        
        epoch_loss = (float)(running_loss / len(train_data))
        epoch_acc = (float)(running_corrects / len(train_data))

        print("train loss:",epoch_loss,"train accuracy:",epoch_acc)

        # model.eval()
        running_loss = 0.0
        running_corrects = 0.0

        for i in range(len(test_data)):
            # im1 = prepare_image(test_data[i][0])
            # im2 = prepare_image(test_data[i][1])
            im1 = test_data[i][0]
            im2 = test_data[i][1]

            label = (float)(test_label[i])

            with torch.set_grad_enabled(False):
                op1 = (model(im1))
                op2 = (model(im2))
                if op1>op2:
                    output_label = 1.0
                else:
                    output_label = 0.0

            # loss = my_loss(op1,op2,output_label)
            loss = my_loss(op1,op2,label)
            running_loss += loss.item()
            if output_label == label:
                running_corrects += 1
        
        epoch_loss = (float)(running_loss / len(test_data))
        epoch_acc = (float)(running_corrects / len(test_data))

        print("val loss:",epoch_loss,"val accuracy:",epoch_acc)

        if epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [0]:
# Batch propagation

def test_model(model, optimizer, scheduler, num_epochs=25):
    # print the validation accuracies

    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    num_epochs = 1
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        model.train()

        running_loss = 0.0
        running_corrects = 0.0

        # model.eval()
        running_loss = 0.0
        running_corrects = 0.0

        for i in range(len(test_data)):
            # im1 = prepare_image(test_data[i][0])
            # im2 = prepare_image(test_data[i][1])
            im1 = test_data[i][0]
            im2 = test_data[i][1]

            label = (float)(test_label[i])

            with torch.set_grad_enabled(False):
                op1 = (model(im1))
                op2 = (model(im2))
                if op1>op2:
                    output_label = 1.0
                else:
                    output_label = 0.0

            # loss = my_loss(op1,op2,output_label)
            loss = my_loss(op1,op2,label)
            running_loss += loss.item()
            if output_label == label:
                running_corrects += 1
        
        epoch_loss = (float)(running_loss / len(test_data))
        epoch_acc = (float)(running_corrects / len(test_data))

        print("val loss:",epoch_loss,"val accuracy:",epoch_acc)

        if epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

for i in range(0,21):

    # print test accuracies

    print((i%6)*2000,"to",((i%6))*2000+2000)
    pdip = np.asarray(pdip_strict)[(i%7)*2000:((i%7)+1)*2000]
    # np.random.shuffle(pdip)

    train_test_mark = int(pdip_train.shape[0]*0.8)

    temp = []
    img_not_found = []
    for i in pdip:
        if (i[0]+".jpg") in img_names and (i[1]+".jpg") in img_names:
            temp.append([int(i[0]),int(i[1])])
        elif (i[0]+".jpg") in img_names:
            img_not_found.append(i[1])
        elif (i[1]+".jpg") in img_names:
            img_not_found.append(i[0])
        else:
            img_not_found.append(i[0])
            img_not_found.append(i[1])
    pdip_train = np.asarray(temp)


    train_data = []
    test_data = []
    train_label = []
    test_label = []

    counter = 0

    counter = 0
    for img_p in pdip_train[train_test_mark:]:
        print(counter)
        counter += 1
        ipins = []
        label = -1
        ipins = [ prepare_image(Image.open("drive/My Drive/Data/Images/"+str(img_p[0])+".jpg") ), prepare_image( Image.open("drive/My Drive/Data/Images/"+str(img_p[1])+".jpg") ) ]
        label = 1
        test_data.append(ipins)
        test_label.append(label)
    print("creating test done")
    train_len = train_test_mark

    train_len=len(train_data)
    val_len = pdip_train.shape[0] - train_test_mark

    model_ft = torchvision.models.resnet50()
    # model.avgpool = nn.AdaptiveAvgPool2d(1) # for any size of the input
    model_ft.fc = torch.nn.Linear(in_features=2048, out_features=1)
    model_ft.load_state_dict(torch.load('drive/My Drive/temp', map_location=device)) 
    model_ft.eval().to(device)
    print()
    num_ftrs = model_ft.fc.in_features
    # model_ft.fc = nn.Linear(num_ftrs, 2)

    model_ft = model_ft.to(device)


    # Observe that all parameters are being optimized
    optimizer_ft = torch.optim.SGD(model_ft.parameters(), lr=0.00001, momentum=0.9)

    # Decay LR by a factor of 0.1 every 7 epochs
    exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.5)

    print(len(train_data),len(test_data))
    test_model(model_ft, optimizer_ft, exp_lr_scheduler,num_epochs=9)           



In [0]:
#this is where we do part by part
pdip = np.asarray(pdip_strict)[:2000]
# np.random.shuffle(pdip)

In [0]:
os.getcwd()

'/content/drive/My Drive/Data/Tweets'

In [0]:
os.chdir("../../../..")

In [0]:
os.getcwd()

'/content'

In [0]:
img_folder_path = "drive/My Drive/Data/Images/"


In [0]:
img_paths = [img_folder_path+"/" + f for f in os.listdir(img_folder_path)]
img_names = [f for f in os.listdir(img_folder_path)]

In [0]:
#make sure both images from each pair are present in the dataset

temp = []
img_not_found = []
for i in pdip:
    if (i[0]+".jpg") in img_names and (i[1]+".jpg") in img_names:
        temp.append([int(i[0]),int(i[1])])
    elif (i[0]+".jpg") in img_names:
        img_not_found.append(i[1])
    elif (i[1]+".jpg") in img_names:
        img_not_found.append(i[0])
    else:
        img_not_found.append(i[0])
        img_not_found.append(i[1])
pdip_train = np.asarray(temp)

In [0]:
# for the train set, shuffle the columns in each row and set the label accordingly so that there is equal distribution of pairs with the left image more popular, and pairs with the right image more popular

train_test_mark = int(pdip_train.shape[0]*0.8)
train_data = []
test_data = []
train_label = []
test_label = []

counter = 0

for img_p in pdip_train[:train_test_mark]:
    if counter%50==0:
        print(counter)
    counter += 1
    ipins = []
    label = -1
    if random.uniform(0,1) > 0.5:
        ipins = [  prepare_image(Image.open("drive/My Drive/Data/Images/"+str(img_p[0])+".jpg") ), prepare_image( Image.open("drive/My Drive/Data/Images/"+str(img_p[1])+".jpg") ) ]
        label = 1
    else:
        ipins = [  prepare_image(Image.open("drive/My Drive/Data/Images/"+str(img_p[1])+".jpg") ), prepare_image( Image.open("drive/My Drive/Data/Images/"+str(img_p[0])+".jpg") ) ]
        label = 0
    train_data.append(ipins)
    train_label.append(label)
    # if counter==2000:
    #     break



In [0]:
print("train done")
#create a test dataset after ensuring all images are present in the dataset
counter = 0
for img_p in pdip_train[train_test_mark:]:
    counter += 1
    if counter % 50 == 0:
        print(counter)
    ipins = []
    label = -1
    ipins = [ prepare_image(Image.open("drive/My Drive/Data/Images/"+str(img_p[0])+".jpg") ), prepare_image( Image.open("drive/My Drive/Data/Images/"+str(img_p[1])+".jpg") ) ]
    label = 1
    test_data.append(ipins)
    test_label.append(label)
print("test done")


In [0]:
train_len = train_test_mark

train_len=len(train_data)
val_len = pdip_train.shape[0] - train_test_mark

In [0]:
#load the pretrained model
model_ft = torchvision.models.resnet50()
# model.avgpool = nn.AdaptiveAvgPool2d(1) # for any size of the input
model_ft.fc = torch.nn.Linear(in_features=2048, out_features=1)
model_ft.load_state_dict(torch.load('drive/My Drive/temp', map_location=device)) 
model_ft.eval().to(device)
print()

In [0]:
#initialize an optimizer and scheduler
num_ftrs = model_ft.fc.in_features
# model_ft.fc = nn.Linear(num_ftrs, 2)

model_ft = model_ft.to(device)


# Observe that all parameters are being optimized
optimizer_ft = torch.optim.SGD(model_ft.parameters(), lr=0.00001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.5)

In [0]:
# probability = 1 --> img1 is popular
# probability = 0 --> img2 is popular

In [0]:
print(len(train_data),len(test_data))
# train the model
model_ft = train_model(model_ft, optimizer_ft, exp_lr_scheduler,num_epochs=9)

In [0]:
torch.save(model_ft.state_dict(),'drive/My Drive/temp')

In [0]:
# perform Batch propagation on the model to fine tune to twitter

for i in range(1,21):
    print((i%6)*2000,"to",((i%6))*2000+2000)
    pdip = np.asarray(pdip_strict)[(i%7)*2000:((i%7)+1)*2000]
    # np.random.shuffle(pdip)

    train_test_mark = int(pdip_train.shape[0]*0.8)

    temp = []
    img_not_found = []
    for i in pdip:
        if (i[0]+".jpg") in img_names and (i[1]+".jpg") in img_names:
            temp.append([int(i[0]),int(i[1])])
        elif (i[0]+".jpg") in img_names:
            img_not_found.append(i[1])
        elif (i[1]+".jpg") in img_names:
            img_not_found.append(i[0])
        else:
            img_not_found.append(i[0])
            img_not_found.append(i[1])
    pdip_train = np.asarray(temp)


    train_data = []
    test_data = []
    train_label = []
    test_label = []

    counter = 0

    for img_p in pdip_train[:train_test_mark]:
        if counter%50==0:
            print(counter)
        counter += 1
        ipins = []
        label = -1
        if random.uniform(0,1) > 0.5:
            ipins = [  prepare_image(Image.open("drive/My Drive/Data/Images/"+str(img_p[0])+".jpg") ), prepare_image( Image.open("drive/My Drive/Data/Images/"+str(img_p[1])+".jpg") ) ]
            label = 1
        else:
            ipins = [  prepare_image(Image.open("drive/My Drive/Data/Images/"+str(img_p[1])+".jpg") ), prepare_image( Image.open("drive/My Drive/Data/Images/"+str(img_p[0])+".jpg") ) ]
            label = 0
        train_data.append(ipins)
        train_label.append(label)
        # if counter==2000:
        #     break
    print("creating train done")

    counter = 0
    for img_p in pdip_train[train_test_mark:]:
        print(counter)
        counter += 1
        ipins = []
        label = -1
        ipins = [ prepare_image(Image.open("drive/My Drive/Data/Images/"+str(img_p[0])+".jpg") ), prepare_image( Image.open("drive/My Drive/Data/Images/"+str(img_p[1])+".jpg") ) ]
        label = 1
        test_data.append(ipins)
        test_label.append(label)
    print("creating test done")
    train_len = train_test_mark

    train_len=len(train_data)
    val_len = pdip_train.shape[0] - train_test_mark

    model_ft = torchvision.models.resnet50()
    # model.avgpool = nn.AdaptiveAvgPool2d(1) # for any size of the input
    model_ft.fc = torch.nn.Linear(in_features=2048, out_features=1)
    model_ft.load_state_dict(torch.load('drive/My Drive/temp', map_location=device)) 
    model_ft.eval().to(device)
    print()
    num_ftrs = model_ft.fc.in_features
    # model_ft.fc = nn.Linear(num_ftrs, 2)

    model_ft = model_ft.to(device)


    # Observe that all parameters are being optimized
    optimizer_ft = torch.optim.SGD(model_ft.parameters(), lr=0.00001, momentum=0.9)

    # Decay LR by a factor of 0.1 every 7 epochs
    exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.5)

    print(len(train_data),len(test_data))
    model_ft = train_model(model_ft, optimizer_ft, exp_lr_scheduler,num_epochs=9)

    torch.save(model_ft.state_dict(),'drive/My Drive/temp')